<a href="https://colab.research.google.com/github/dasneelay/360Project/blob/main/HfPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datasets
import torch
import transformers

In [ ]:
ds_train = datasets.load_dataset("huggingface-course/codeparrot-ds-train", split="train")
ds_valid = datasets.load_dataset("huggingface-course/codeparrot-ds-valid", split="validation")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
raw_datasets = datasets.DatasetDict(
    {
        "train": ds_train.shuffle(),
        "valid": ds_valid.shuffle()
    }
)

raw_datasets

DatasetDict({
    train: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 606720
    })
    valid: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 3322
    })
})

In [ ]:
for feature in raw_datasets["train"][0]:
  print(f"{feature}: {raw_datasets['train'][0][feature][:200]}")

repo_name: alalbiol/trading-with-python
path: historicDataDownloader/historicDataDownloader.py
copies: 77
size: 4526
content: '''
Created on 4 aug. 2012
Copyright: Jev Kuznetsov
License: BSD

a module for downloading historic data from IB

'''
import ib
import pandas
from ib.ext.Contract import Contract
from ib.op
license: bsd-3-clause


In [ ]:
context_length = 128

tokenizer = transformers.AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")

outputs = tokenizer(
    raw_datasets["train"][:2]["content"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True,
    return_length=True
)
print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

NameError: name 'transformers' is not defined